In [ ]:
!nvidia-smi
# # memory footprint support libraries/code
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize
# import psutil
# import humanize
# import os
# import GPUtil as GPU
# GPUs = GPU.getGPUs()
# # XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]
# def printm():
#  process = psutil.Process(os.getpid())
#  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
#  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
# printm()

# you'll need around 16.28 GB GPU RAM, if lower reset and pray for rng gods

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir /content/drive/My\ Drive/ModelsH5

!git clone https://github.com/abbsimoga/CardDetection
!mkdir video_cards

!mkdir generated_images
!mkdir generated_images_xml

!mkdir generated_images_validation
!mkdir generated_images_xml_validation

!wget https://www.robots.ox.ac.uk/~vgg/data/dtd/download/dtd-r1.0.1.tar.gz
!tar xf dtd-r1.0.1.tar.gz
!rm /content/dtd-r1.0.1.tar.gz

!git clone https://github.com/AIWintermuteAI/aXeleRate.git

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/My Drive/ModelsH5’: File exists
Cloning into 'CardDetection'...
remote: Enumerating objects: 75, done.
remote: Total 75 (delta 0), reused 0 (delta 0), pack-reused 75
Unpacking objects: 100% (75/75), done.
Checking out files: 100% (54/54), done.
--2020-07-19 07:37:01--  https://www.robots.ox.ac.uk/~vgg/data/dtd/download/dtd-r1.0.1.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting 

In [ ]:
%tensorflow_version 1.x
import sys
sys.path.append('/content/aXeleRate')
from axelerate import setup_training, setup_inference

import os
from glob import glob 
import xml.etree.cElementTree as ET

import math
import numpy as np
import random

import imgaug.augmenters as iaa
from imgaug.augmentables import Keypoint, KeypointsOnImage
import imutils
import cv2
from google.colab.patches import cv2_imshow

from keras import backend as K

cardW=57
cardH=87
# cornerXmin=2
# cornerXmax=10.5
# cornerYmin=2.5
# cornerYmax=23
cornerXmin=3.25
cornerXmax=8.5
cornerYmin=3.25
cornerYmax=18.5

zoom=4
cardW*=zoom
cardH*=zoom
cornerXmin=int(cornerXmin*zoom)
cornerXmax=int(cornerXmax*zoom)
cornerYmin=int(cornerYmin*zoom)
cornerYmax=int(cornerYmax*zoom)

refCard=np.array([[0,0],[cardW,0],[cardW,cardH],[0,cardH]],dtype=np.float32)
refCardRot=np.array([[cardW,0],[cardW,cardH],[0,cardH],[0,0]],dtype=np.float32)
refCornerHL=np.array([[cornerXmin,cornerYmin],[cornerXmax,cornerYmin],[cornerXmax,cornerYmax],[cornerXmin,cornerYmax]],dtype=np.float32)
refCornerLR=np.array([[cardW-cornerXmax,cardH-cornerYmax],[cardW-cornerXmin,cardH-cornerYmax],[cardW-cornerXmin,cardH-cornerYmin],[cardW-cornerXmax,cardH-cornerYmin]],dtype=np.float32)
# refCorners=np.array([refCornerHL,refCornerLR])

refCornerHR=np.array([[cardW-cornerXmax,cornerYmin],[cardW-cornerXmin,cornerYmin],[cardW-cornerXmin,cornerYmax],[cardW-cornerXmax,cornerYmax]],dtype=np.float32) # 123
refCornerLL=np.array([[cornerXmin,cardH-cornerYmax],[cornerXmax,cardH-cornerYmax],[cornerXmax,cardH-cornerYmin],[cornerXmin,cardH-cornerYmin]],dtype=np.float32) # 123
refCorners=np.array([refCornerHL,refCornerHR,refCornerLL,refCornerLR]) # 123

bord_size=2
alphamask=np.ones((cardH,cardW),dtype=np.uint8)*255
cv2.rectangle(alphamask,(0,0),(cardW-1,cardH-1),0,bord_size)
cv2.line(alphamask,(bord_size*3,0),(0,bord_size*3),0,bord_size)
cv2.line(alphamask,(cardW-bord_size*3,0),(cardW,bord_size*3),0,bord_size)
cv2.line(alphamask,(0,cardH-bord_size*3),(bord_size*3,cardH),0,bord_size)
cv2.line(alphamask,(cardW-bord_size*3,cardH),(cardW,cardH-bord_size*3),0,bord_size)

card_suits = ['s','h','d','c']
card_values = ['A','K','Q','J','10','9','8','7','6','5','4','3','2']

model_path = ""

TensorFlow 1.x selected.


Using TensorFlow backend.


In [ ]:
def showmask(image):
    # cv2.line(image, (int(cornerXmin), 0), (int(cornerXmin), int(cornerYmin)+int(cornerYmax)), 0, thickness=1)
    # cv2.line(image, (int(cornerXmax), 0), (int(cornerXmax), int(cornerYmin)+int(cornerYmax)), 0, thickness=1)
    # cv2.line(image, (0, int(cornerYmin)), (int(cornerXmin)+int(cornerXmax), int(cornerYmin)), 0, thickness=1)
    # cv2.line(image, (0, int(cornerYmax)), (int(cornerXmin)+int(cornerXmax), int(cornerYmax)), 0, thickness=1)

    # cv2.line(image, (cardW-int(cornerXmin), cardH-int(cornerYmin)-int(cornerYmax)), (cardW-int(cornerXmin), cardH), 0, thickness=1)
    # cv2.line(image, (cardW-int(cornerXmax), cardH-int(cornerYmin)-int(cornerYmax)), (cardW-int(cornerXmax), cardH), 0, thickness=1)
    # cv2.line(image, (cardW-int(cornerXmin)-int(cornerXmax), cardH-int(cornerYmin)), (cardW, cardH-int(cornerYmin)), 0, thickness=1)
    # cv2.line(image, (cardW-int(cornerXmin)-int(cornerXmax), cardH-int(cornerYmax)), (cardW, cardH-int(cornerYmax)), 0, thickness=1)

    hullHL=find_hull(image, refCornerHL)
    hullLR=find_hull(image, refCornerLR)

    hullHR=find_hull(image, refCornerHR) # 123
    hullLL=find_hull(image, refCornerLL) # 123

    cv2.line(image,(int(cornerXmin), 0), (int(cornerXmin), cardH), 0, thickness=1)
    cv2.line(image,(int(cornerXmax), 0), (int(cornerXmax), cardH), 0, thickness=1)
    cv2.line(image,(cardW-int(cornerXmin), 0), (cardW-int(cornerXmin), cardH), 0, thickness=1)
    cv2.line(image,(cardW-int(cornerXmax), 0), (cardW-int(cornerXmax), cardH), 0, thickness=1)

    cv2.line(image,(0, int(cornerYmin)), (cardW, int(cornerYmin)), 0, thickness=1)
    cv2.line(image,(0, int(cornerYmax)), (cardW, int(cornerYmax)), 0, thickness=1)
    cv2.line(image,(0, cardH-int(cornerYmin)), (cardW, cardH-int(cornerYmin)), 0, thickness=1)
    cv2.line(image,(0, cardH-int(cornerYmax)), (cardW, cardH-int(cornerYmax)), 0, thickness=1)

    cv2.drawContours(image, find_hull(image, refCornerHL),0,(0,255,0),2)
    cv2.drawContours(image, find_hull(image, refCornerLR),0,(0,255,0),2)
    cv2.drawContours(image, find_hull(image, refCornerHR),0,(0,255,0),2)
    cv2.drawContours(image, find_hull(image, refCornerLL),0,(0,255,0),2)

    cv2.drawContours(image,[hullHL],0,(0,255,0),2)
    cv2.drawContours(image,[hullLR],0,(0,255,0),2)
    cv2.drawContours(image,[hullHR],0,(0,255,0),2)
    cv2.drawContours(image,[hullLL],0,(0,255,0),2)

# cv2.drawContours(strange_cnt,[c],0,255,2)
 
    cv2_imshow(image)
    
def extract_card(img, output_fn, min_focus=120):
    # focus=cv2.Laplacian(img, cv2.CV_64F).var()
    # if focus < min_focus:
    #     return False, None

    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray=cv2.bilateralFilter(gray,11,17,17)
    edge=cv2.Canny(gray,30,200)
    cnts, _ = cv2.findContours(edge.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnt = sorted(cnts, key = cv2.contourArea)[-1]
    
    # cv2_imshow(gray)
    # cv2_imshow(edge)
    # edge_bgr=cv2.cvtColor(edge,cv2.COLOR_GRAY2BGR)
    # cv2.drawContours(edge_bgr,[box],0,(0,0,255),3)
    # cv2.drawContours(edge_bgr,[cnt],0,(0,255,0),-1)
    # cv2.imshow("Contour with biggest area",edge_bgr)
    # if valid:
    #     cv2.imshow("Alphachannel",alphachannel)
    #     cv2.imshow("Extracted card",imgwarp)

    rect=cv2.minAreaRect(cnt)
    box=cv2.boxPoints(rect)
    box=np.int0(box)
    areaCnt=cv2.contourArea(cnt)
    areaBox=cv2.contourArea(box)
    valid=areaCnt/areaBox>0.95
    
    if valid:
        ((xr,yr),(wr,hr),thetar)=rect

        if wr>hr:
            Mp=cv2.getPerspectiveTransform(np.float32(box),refCard)
        else:
            Mp=cv2.getPerspectiveTransform(np.float32(box),refCardRot)
            
        imgwarp=cv2.warpPerspective(img,Mp,(cardW,cardH))
        imgwarp=cv2.cvtColor(imgwarp,cv2.COLOR_BGR2BGRA)
        
        cnta=cnt.reshape(1,-1,2).astype(np.float32)
        cntwarp=cv2.perspectiveTransform(cnta,Mp)
        cntwarp=cntwarp.astype(np.int)
        
        alphachannel=np.zeros(imgwarp.shape[:2],dtype=np.uint8)
        cv2.drawContours(alphachannel,cntwarp,0,255,-1)
        alphachannel=cv2.bitwise_and(alphachannel,alphamask)
        imgwarp[:,:,3]=alphachannel
        
        if output_fn != None:
            cv2.imwrite(output_fn, imgwarp)
    else:
        return False, None
    return valid, imgwarp

def extract_cards_from_video(video_dir, output_dir, keep_ratio=5, min_focus=120):
    cap=cv2.VideoCapture(video_dir)

    image_number=0    
    frame_number=0
    while True:
        ret, img=cap.read()
        if not ret:
            break
        
        if frame_number%keep_ratio == 0:
            valid, card_img = extract_card(img, output_dir+"{}.png".format(str(image_number).zfill(8)), min_focus)
            if valid:
                image_number += 1
        frame_number+=1

def same_amounts_of_cards(card_stem_dir):
    file_amounts = []
    for card_suit in card_suits:
        for card_value in card_values:
            cards_dir = card_stem_dir+card_suit+"/"+card_value+"/*"
            list_of_files = glob(cards_dir)
            file_amount = len(list_of_files)
            # latest_file = max(list_of_files, key=os.path.getctime)
            # file_amount = int(latest_file[-12:-4])+1
            file_amounts.append(file_amount)
    card_min_amount = min(file_amounts)
    print("card_min_amount:", card_min_amount)
    remove_amount_from_dir = [x-card_min_amount for x in file_amounts]
    rafd_index = 0
    for card_suit in card_suits:
        for card_value in card_values:
            cards_dir = card_stem_dir+card_suit+"/"+card_value
            for cfile in random.sample(os.listdir(cards_dir), remove_amount_from_dir[rafd_index]):
                os.remove(card_stem_dir+card_suit+"/"+card_value+"/"+cfile)
            list_of_files = glob(cards_dir+"/*")
            file_amount = len(list_of_files)
            print("prior file amount:", file_amounts[rafd_index], "removed:", remove_amount_from_dir[rafd_index], "file amount:", file_amount)
            rafd_index +=1

def extract_cards_from_video_folder(video_stem_dir, card_stem_dir, keep_ratio=5, min_focus=120):
    for card_suit in card_suits:
        for card_value in card_values:
            card_output_dir = card_stem_dir+card_suit+"/"+card_value+"/"
            video_dir = video_stem_dir + card_suit + card_value + ".MOV" #".avi"
            if not os.path.isdir(card_output_dir):
                os.makedirs(card_output_dir)
            extract_cards_from_video(video_dir, card_output_dir, keep_ratio, min_focus)

def find_hull(img, corner):
    kernel = np.ones((3,3),np.uint8)
    corner=corner.astype(np.int)

    x1=int(corner[0][0])
    y1=int(corner[0][1])
    x2=int(corner[2][0])
    y2=int(corner[2][1])
    w=x2-x1
    h=y2-y1
    zone=img[y1:y2,x1:x2].copy()

    strange_cnt=np.zeros_like(zone)
    gray=cv2.cvtColor(zone,cv2.COLOR_BGR2GRAY)
    thld=cv2.Canny(gray,30,200)
    thld = cv2.dilate(thld,kernel,iterations=1)

    contours,_=cv2.findContours(thld.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

    min_area=30 # We will reject contours with small area. TWEAK, 'zoom' dependant
    min_solidity=0.3 # Reject contours with a low solidity. TWEAK
    
    concat_contour=None
    
    # ok=True
    for c in contours:
        area=cv2.contourArea(c)

        hull = cv2.convexHull(c)
        hull_area = cv2.contourArea(hull)
        solidity = float(area)/hull_area

        M=cv2.moments(c)
        cx=int(M['m10']/M['m00'])
        cy=int(M['m01']/M['m00'])
        
        if area >= min_area and abs(w/2-cx)<w*0.3 and abs(h/2-cy)<h*0.4 and solidity>min_solidity:
            if concat_contour is None:
                concat_contour=c
            else:
                concat_contour=np.concatenate((concat_contour,c))    
     
    # if concat_contour is not None:
    hull=cv2.convexHull(concat_contour)
        # hull_area=cv2.contourArea(hull)

        # min_hull_area=940 # TWEAK, deck and 'zoom' dependant
        # max_hull_area=2120 # TWEAK, deck and 'zoom' dependant

        # if hull_area < min_hull_area or hull_area > max_hull_area:
        #     ok=False
    hull_in_img=hull+corner[0]
    # else:
    #     ok=False

    # if not ok:
    #     # print("AAAAAAAAAAAAAAAAAAA")
    #     return None
    
    return hull_in_img

def get_random_background():
    return cv2.imread(random.choice(glob("/content/dtd/images/*/*.jpg")))

def get_random_card(card_pick_dir, suit):
    # card_name = random.choice(glob("/content/video_cards/*/*/*.png"))
    card_name = random.choice(glob(card_pick_dir+"/*/*/*.png")) if suit == "all" else random.choice(glob(card_pick_dir+"/"+suit+"/*/*.png"))
    return cv2.imread(card_name, cv2.IMREAD_UNCHANGED), card_name[-17:-12].replace("/","")

def create_root(file_prefix, width, height):
    root = ET.Element("annotations")
    ET.SubElement(root, "filename").text = "{}.JPEG".format(file_prefix)
    ET.SubElement(root, "folder").text = "images"
    size = ET.SubElement(root, "size")
    ET.SubElement(size, "width").text = str(width)
    ET.SubElement(size, "height").text = str(height)
    ET.SubElement(size, "depth").text = "3"
    return root
 
def create_object_annotation(root, voc_labels):
    for voc_label in voc_labels:
        obj = ET.SubElement(root, "object")
        ET.SubElement(obj, "name").text = voc_label[0]
        ET.SubElement(obj, "pose").text = "Unspecified"
        ET.SubElement(obj, "truncated").text = str(0)
        ET.SubElement(obj, "difficult").text = str(0)
        bbox = ET.SubElement(obj, "bndbox")
        ET.SubElement(bbox, "xmin").text = str(voc_label[1])
        ET.SubElement(bbox, "ymin").text = str(voc_label[2])
        ET.SubElement(bbox, "xmax").text = str(voc_label[3])
        ET.SubElement(bbox, "ymax").text = str(voc_label[4])
    return root

def create_training_image(card_dir, suit, destination_dir_img, destination_dir_xml, min_rotation_allowed=1, max_rotation_allowed=360, scale_min_allowed=0.25, scale_max_allowed=0.9, training_images_amount=0, boundary_leeway=3, cover_max=0.25, max_cards_allowed=3, preview=False):
    img_background = get_random_background()

    mask=np.zeros((img_background.shape[0],img_background.shape[1],4),dtype=np.uint8)

    cards_on_canvas=[]

    for _ in range(random.randint(1, max_cards_allowed)):
        
        img_card, card_name = get_random_card(card_pick_dir,suit)
        img_card = cv2.cvtColor(img_card, cv2.COLOR_RGB2RGBA)

        hullHL=find_hull(img_card, refCornerHL)
        hullLR=find_hull(img_card, refCornerLR)

        hullHR=find_hull(img_card, refCornerHR) # 123
        hullLL=find_hull(img_card, refCornerLL) # 123

        keypointsHL = [Keypoint(x=point[0][0], y=point[0][1]) for point in hullHL]
        keypointsLR = [Keypoint(x=point[0][0], y=point[0][1]) for point in hullLR]
        
        keypointsHR = [Keypoint(x=point[0][0], y=point[0][1]) for point in hullHR] # 123
        keypointsLL = [Keypoint(x=point[0][0], y=point[0][1]) for point in hullLL] # 123

        kpsHL = KeypointsOnImage(keypointsHL, shape=(img_card.shape[0], img_card.shape[1]))
        kpsLR = KeypointsOnImage(keypointsLR, shape=(img_card.shape[0], img_card.shape[1]))

        kpsHR = KeypointsOnImage(keypointsHR, shape=(img_card.shape[0], img_card.shape[1])) # 123
        kpsLL = KeypointsOnImage(keypointsLL, shape=(img_card.shape[0], img_card.shape[1])) # 123

        rotation = random.randint(min_rotation_allowed, max_rotation_allowed)

        seq = iaa.Affine(rotate=rotation, fit_output=True)
        # image_aug, kps_aug = seq(image=img_card, keypoints=[kpsHL,kpsLR])
        image_aug, kps_aug = seq(image=img_card, keypoints=[kpsHL,kpsLR,kpsHR,kpsLL])

        acceptable_scale = min(img_background.shape[0]/image_aug.shape[0], img_background.shape[1]/image_aug.shape[1])
        max_accept_scale = acceptable_scale * scale_max_allowed
        min_accept_scale = acceptable_scale * scale_min_allowed
        scale = random.uniform(min_accept_scale, max_accept_scale)

        seqq = iaa.Affine(scale=scale, fit_output=True)
        image_aug, kps_aug = seqq(image=image_aug, keypoints=kps_aug)

        y_offset = random.randint(0, img_background.shape[0]-image_aug.shape[0])
        x_offset = random.randint(0, img_background.shape[1]-image_aug.shape[1])
            
        y1, y2 = y_offset, y_offset + image_aug.shape[0]
        x1, x2 = x_offset, x_offset + image_aug.shape[1]
        
        for point, kp in zip(hullHL, kps_aug[0].keypoints):
            point[0][0]=int(kp.x+x_offset)
            point[0][1]=int(kp.y+y_offset)
          
        for point, kp in zip(hullLR, kps_aug[1].keypoints):
            point[0][0]=int(kp.x+x_offset)
            point[0][1]=int(kp.y+y_offset)

        for point, kp in zip(hullHR, kps_aug[2].keypoints): # 123
            point[0][0]=int(kp.x+x_offset)
            point[0][1]=int(kp.y+y_offset)
          
        for point, kp in zip(hullLL, kps_aug[3].keypoints): # 123
            point[0][0]=int(kp.x+x_offset)
            point[0][1]=int(kp.y+y_offset)

        alpha_s = image_aug[:, :, 3] / 255.0
        alpha_l = 1.0 - alpha_s

        blank_card = np.zeros_like(img_background)
        
        image_trans_mask = image_aug.copy()

        trans_mask = image_aug[:,:,3] == 255
        image_trans_mask[trans_mask] = [255, 255, 255, 255]

        for c in range(0, 3):
            blank_card[y1:y2, x1:x2, c] = (alpha_s * image_trans_mask[:, :, c] + alpha_l * blank_card[y1:y2, x1:x2, c])

        right_marks = []
        for marks in cards_on_canvas:
            blank_marks = np.zeros_like(img_background)
            cv2.drawContours(blank_marks, [marks[5]], -1, (255,255,255,255), -1)
            intersection = np.logical_and(blank_card, blank_marks)
            mark_pixel_amount = (blank_marks==255).sum()
            intersected_amount = (intersection==True).sum()
            if intersected_amount/mark_pixel_amount < cover_max:
                right_marks.append(marks)

        cards_on_canvas = right_marks

        for c in range(0, 3):
            img_background[y1:y2, x1:x2, c] = (alpha_s * image_aug[:, :, c] + alpha_l * img_background[y1:y2, x1:x2, c])

        x1,y1,w1,h1 = cv2.boundingRect(hullHL)
        x2,y2,w2,h2 = cv2.boundingRect(hullLR)
        
        x3,y3,w3,h3 = cv2.boundingRect(hullHR) # 123
        x4,y4,w4,h4 = cv2.boundingRect(hullLL) # 123

        cards_on_canvas.append([card_name, x1-boundary_leeway, y1-boundary_leeway, x1+w1+boundary_leeway, y1+h1+boundary_leeway, hullHL])
        cards_on_canvas.append([card_name, x2-boundary_leeway, y2-boundary_leeway, x2+w2+boundary_leeway, y2+h2+boundary_leeway, hullLR])

        cards_on_canvas.append([card_name, x3-boundary_leeway, y3-boundary_leeway, x3+w3+boundary_leeway, y3+h3+boundary_leeway, hullHR]) # 123
        cards_on_canvas.append([card_name, x4-boundary_leeway, y4-boundary_leeway, x4+w4+boundary_leeway, y4+h4+boundary_leeway, hullLL]) # 123

    file_prefix = str(training_images_amount).zfill(8)
    root = create_root(file_prefix, img_background.shape[0], img_background.shape[1])
    root = create_object_annotation(root, cards_on_canvas)
    tree = ET.ElementTree(root) 

    cv2.imwrite("{}/{}.JPEG".format(destination_dir_img, file_prefix), img_background)
    tree.write("{}/{}.xml".format(destination_dir_xml, file_prefix))
    if preview:
        for marks in cards_on_canvas:
            cv2.rectangle(img_background, (marks[1],marks[2]), (marks[3],marks[4]), (255,0,0), 1)
            cv2.drawContours(img_background,[marks[5]],0,(0,255,0),1)

        cv2_imshow(img_background)

In [ ]:
# extract_cards_from_video_folder("/content/CardDetection/videos/", "/content/video_cards/", 1, 120)
# same_amounts_of_cards("/content/video_cards/")
# card_pick_dir = "/content/video_cards/"

In [ ]:
if not os.path.isdir("/content/drive/My Drive/video_cards"):
    extract_cards_from_video_folder("/content/CardDetection/videos/", "/content/video_cards/", 1, 120)
    same_amounts_of_cards("/content/video_cards/")
    !cp -a /content/video_cards /content/drive/My\ Drive/
else:
    !cp -a /content/drive/My\ Drive/video_cards /content/

card_pick_dir = "/content/video_cards/"

In [ ]:
# # for _ in range(10):
# card, cardname=get_random_card(card_pick_dir, "all")
# print(cardname)
# showmask(card)

In [ ]:
#TEST
suit = "all"
image_existance=0
for image_amount in range(10):
    if (image_amount+1)%1000 == 0: print(image_amount+1)
    create_training_image(
        card_pick_dir, suit, "/content/generated_images", "/content/generated_images_xml",
        min_rotation_allowed=1, max_rotation_allowed=360,
        scale_min_allowed=0.4, scale_max_allowed=0.99,
        training_images_amount=image_amount+image_existance,
        boundary_leeway=0, cover_max=0.3,
        max_cards_allowed=4, preview=True
    )

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# for suit in card_suits:
#     print("creating images:", suit)

#     image_existance=0
#     for image_amount in range(2500):
#         if (image_amount+1)%1000 == 0: print(image_amount+1)
#         create_training_image(
#             card_pick_dir, suit, "/content/generated_images", "/content/generated_images_xml",
#             min_rotation_allowed=1, max_rotation_allowed=360,
#             scale_min_allowed=0.9, scale_max_allowed=0.99,
#             training_images_amount=image_amount+image_existance,
#             boundary_leeway=0, cover_max=0.25,
#             max_cards_allowed=1
#         )
#     image_existance=0
#     for image_amount in range(100):
#         create_training_image(
#             card_pick_dir, suit, "/content/generated_images_validation", "/content/generated_images_xml_validation",
#             min_rotation_allowed=1, max_rotation_allowed=360,
#             scale_min_allowed=0.9, scale_max_allowed=0.99,
#             training_images_amount=image_amount+image_existance,
#             boundary_leeway=0, cover_max=0.25,
#             max_cards_allowed=1
#         )
    
#     config = {
#         "model":{
#             "type":             "Detector",
#             "architecture":     "MobileNet1_0",
#             "input_size":       1664,
#             "anchors":          [0.57273, 0.677385, 1.87446, 2.06253, 3.33843, 5.47434, 7.88282, 3.52778, 9.77052, 9.16828],
#             "labels":           ["sA","sK","sQ","sJ","s10","s9","s8","s7","s6","s5","s4","s3","s2","hA","hK","hQ","hJ","h10","h9","h8","h7","h6","h5","h4","h3","h2","dA","dK","dQ","dJ","d10","d9","d8","d7","d6","d5","d4","d3","d2","cA","cK","cQ","cJ","c10","c9","c8","c7","c6","c5","c4","c3","c2"],
#             "coord_scale" : 		1.0,
#             "class_scale" : 		1.0,
#             "object_scale" : 		5.0,
#             "no_object_scale" : 1.0
#         },
#         "weights" : {
#             "full":   				  model_path,
#             "backend":   		    "imagenet"
#         },
#         "train" : {
#             "actual_epoch":         1,
#             "train_image_folder":   "generated_images",
#             "train_annot_folder":   "generated_images_xml",
#             "train_times":          1,
#             "valid_image_folder":   "generated_images_validation",
#             "valid_annot_folder":   "generated_images_xml_validation",
#             "valid_times":          1,
#             "valid_metric":         "mAP",
#             "batch_size":           1,
#             "learning_rate":        1e-4,
#             "saved_folder":   			F"/content/drive/My Drive/ModelsH5",
#             "first_trainable_layer":"",
#             "augumentation":				False,
#             "is_only_detect" : 		  False
#         },
#         "converter" : {
#             "type":   				"k210"
#         }
#     }

#     K.clear_session()
#     model_path = setup_training(config_dict=config)

In [ ]:
model_path="/content/drive/My Drive/ModelsH5/2020-07-17_14-49-24/YOLO_best_mAP.h5"

In [ ]:
# !nvidia-smi
# !ps -aux|grep python
# !kill -9
# !nvidia-smi
print("model_path:",model_path)
config = {
    "model":{
        "type":             "Detector",
        "architecture":     "MobileNet1_0",
        "input_size":       1664,
        "anchors":          [0.57273, 0.677385, 1.87446, 2.06253, 3.33843, 5.47434, 7.88282, 3.52778, 9.77052, 9.16828],
        "labels":           ["sA","sK","sQ","sJ","s10","s9","s8","s7","s6","s5","s4","s3","s2","hA","hK","hQ","hJ","h10","h9","h8","h7","h6","h5","h4","h3","h2","dA","dK","dQ","dJ","d10","d9","d8","d7","d6","d5","d4","d3","d2","cA","cK","cQ","cJ","c10","c9","c8","c7","c6","c5","c4","c3","c2"],
        "coord_scale" : 		1.0,
        "class_scale" : 		1.0,
        "object_scale" : 		5.0,
        "no_object_scale" : 1.0
    },
    "weights" : {
        "full":   				  model_path,
        "backend":   		    "imagenet"
    },
    "train" : {
        "actual_epoch":         10,
        "train_image_folder":   "generated_images",
        "train_annot_folder":   "generated_images_xml",
        "train_times":          1,
        "valid_image_folder":   "generated_images_validation",
        "valid_annot_folder":   "generated_images_xml_validation",
        "valid_times":          1,
        "valid_metric":         "mAP",
        "batch_size":           1,
        "learning_rate":        1e-5,
        "saved_folder":   			F"/content/drive/My Drive/ModelsH5",
        "first_trainable_layer":"",
        "augumentation":				False,
        "is_only_detect" : 		  False
    },
    "converter" : {
        "type":   				"k210"
    }
}

K.clear_session()
model_path = setup_training(config_dict=config)

print("model_path",model_path)

model_path: 
Project folder /content/drive/My Drive/ModelsH5 already exists. Creating a folder for new training session.
7397376/7392744 [==============================] - 2s 0us/step
/content/aXeleRate/axelerate/networks/common_utils/ncc_linux_x86_64.tar.xz
['sA', 'sK', 'sQ', 'sJ', 's10', 's9', 's8', 's7', 's6', 's5', 's4', 's3', 's2', 'hA', 'hK', 'hQ', 'hJ', 'h10', 'h9', 'h8', 'h7', 'h6', 'h5', 'h4', 'h3', 'h2', 'dA', 'dK', 'dQ', 'dJ', 'd10', 'd9', 'd8', 'd7', 'd6', 'd5', 'd4', 'd3', 'd2', 'cA', 'cK', 'cQ', 'cJ', 'c10', 'c9', 'c8', 'c7', 'c6', 'c5', 'c4', 'c3', 'c2']
17227776/17225924 [==============================] - 2s 0us/step
Successfully loaded imagenet backend weights
Failed to load pre-trained weights for the whole model. It might be because you didn't specify any or the weight file cannot be found
Current training session folder is /content/drive/My Drive/ModelsH5/2020-07-19_08-57-14


Model: "model_1"
_________________________________________________________________
Layer (

In [ ]:
# !rm -rf generated_images
# !rm -rf generated_images_xml

# !mkdir generated_images
# !mkdir generated_images_xml
suit = "all"

print("creating images:",suit)

image_existance=0
for image_amount in range(10000):
    if (image_amount+1)%1000 == 0: print(image_amount+1)
    create_training_image(
        card_pick_dir, suit, "/content/generated_images", "/content/generated_images_xml",
        min_rotation_allowed=1, max_rotation_allowed=360,
        scale_min_allowed=0.4, scale_max_allowed=0.99,
        training_images_amount=image_amount+image_existance,
        boundary_leeway=0, cover_max=0.3,
        max_cards_allowed=4, preview=False
    )
image_existance=0
for image_amount in range(500):
    create_training_image(
        card_pick_dir, suit, "/content/generated_images_validation", "/content/generated_images_xml_validation",
        min_rotation_allowed=1, max_rotation_allowed=360,
        scale_min_allowed=0.4, scale_max_allowed=0.99,
        training_images_amount=image_amount+image_existance,
        boundary_leeway=0, cover_max=0.3,
        max_cards_allowed=4, preview=False
    )

creating images: all
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [ ]:
K.clear_session()
setup_inference(config, model_path)

Folder /content/drive/My Drive/ModelsH5/2020-07-17_14-49-24/Inference_results is created.
Loading pre-trained weights for the whole model:  /content/drive/My Drive/ModelsH5/2020-07-17_14-49-24/YOLO_best_mAP.h5


/content/aXeleRate/axelerate/networks/yolo/backend/decoder.py:58: RuntimeWarning: overflow encountered in exp
  return 1. / (1. + np.exp(-x))


4-boxes are detected. /content/drive/My Drive/ModelsH5/2020-07-17_14-49-24/Inference_results/00000000.JPEG saved.
/content/drive/My Drive/ModelsH5/2020-07-17_14-49-24/Inference_results/00000000.JPEG
detect_idx: 0, true_idx: 0, matching-score: 0.8423724901915532
detect_idx: 1, true_idx: 2, matching-score: 0.918918918918919
detect_idx: 2, true_idx: 3, matching-score: 0.9736842105263158
detect_idx: 3, true_idx: 1, matching-score: 0.9377298357440549
7-boxes are detected. /content/drive/My Drive/ModelsH5/2020-07-17_14-49-24/Inference_results/00000001.JPEG saved.
/content/drive/My Drive/ModelsH5/2020-07-17_14-49-24/Inference_results/00000001.JPEG
detect_idx: 0, true_idx: None, matching-score: 0
detect_idx: 1, true_idx: 2, matching-score: 0.8847736625514403
detect_idx: 2, true_idx: None, matching-score: 0
detect_idx: 3, true_idx: 0, matching-score: 0.8211508553654744
detect_idx: 4, true_idx: 1, matching-score: 0.7337526205450734
detect_idx: 5, true_idx: 3, matching-score: 0.9021479713603818
d